<a href="https://colab.research.google.com/github/pranitha2144/SignLanguagetoText/blob/main/Experiment0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
sz=128

In [11]:
# Initializing the CNN
classifier = Sequential()

In [12]:
# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [13]:
# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2

# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Step 2 - Preparing the train/test data and training the model
classifier.summary()
# Code copied from - https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               3686528   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [59]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Major project- IV-II/train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Major project- IV-II/test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 
classifier.fit(
        training_set,
        steps_per_epoch=1200, # No of images in training set
        epochs=5,
        validation_data=test_set,
        validation_steps=40)# No

Found 12845 images belonging to 27 classes.
Found 4268 images belonging to 27 classes.
Epoch 1/5
1200/1200 [==============================] - 220s 183ms/step - loss: 0.2745 - accuracy: 0.9113 - val_loss: 0.0441 - val_accuracy: 0.9925
Epoch 2/5
1200/1200 [==============================] - 219s 183ms/step - loss: 0.2458 - accuracy: 0.9195 - val_loss: 0.0117 - val_accuracy: 0.9975
Epoch 3/5
1200/1200 [==============================] - 219s 183ms/step - loss: 0.2396 - accuracy: 0.9238 - val_loss: 0.0239 - val_accuracy: 0.9925
Epoch 4/5
1200/1200 [==============================] - 218s 181ms/step - loss: 0.2259 - accuracy: 0.9282 - val_loss: 0.0123 - val_accuracy: 0.9950
Epoch 5/5
1200/1200 [==============================] - 219s 183ms/step - loss: 0.2078 - accuracy: 0.9350 - val_loss: 0.0031 - val_accuracy: 1.0000


In [69]:
#Confution Matrix and Classification Report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
test_steps_per_epoch = np.math.ceil(test_set.samples / test_set.batch_size)

predictions = classifier.predict(test_set, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

In [67]:
true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys())   

In [78]:
predicted_classes[1]

8

In [62]:
from sklearn import metrics
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    
metrics.accuracy_score(true_classes,predicted_classes)

              precision    recall  f1-score   support

           0       0.03      0.03      0.03       154
           A       0.03      0.03      0.03       156
           B       0.04      0.04      0.04       157
           C       0.03      0.03      0.03       156
           D       0.05      0.05      0.05       155
           E       0.03      0.03      0.03       156
           F       0.06      0.06      0.06       165
           G       0.03      0.03      0.03       153
           H       0.02      0.02      0.02       165
           I       0.06      0.06      0.06       157
           J       0.02      0.02      0.02       166
           K       0.02      0.02      0.02       164
           L       0.02      0.02      0.02       155
           M       0.02      0.02      0.02       165
           N       0.07      0.07      0.07       163
           O       0.02      0.02      0.02       167
           P       0.04      0.05      0.05       155
           Q       0.04    

0.03725398313027179

In [58]:
import os
import numpy as np
from keras.preprocessing import image
actual=[]
predicted=[]
classes= ['0','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
data_path='/content/drive/MyDrive/Major project- IV-II/test'
data_dir_list=os.listdir(data_path)
for dataset in data_dir_list:
  print(dataset)
  img_list=os.listdir(data_path+'/'+dataset)
  correctCount=0
  wrongCount=0
  for img in img_list:
    img_path=data_path+'/'+dataset+'/'+img
    input_image = image.load_img(img_path,target_size=(sz,sz))
    output = classifier.predict(input_image)
    ans=classes[np.argmax(output)]
    if( ans == dataset):
      correctCount+=1
    else:
      wrongCount+=1
    actual.append(img[:6])
    predicted.append(ans)
  print(dataset,"correct Count {} WrongCount {}".format(correctCount,wrongCount))

E


ValueError: ignored